# VRAM Efficiency Analysis Demo

This notebook demonstrates the use of the `plot_vram_efficiency` method in the `EfficiencyAnalysis` class. The method allows for customizable VRAM efficiency plotting for selected users over a specified date range.

In [ ]:
# Import required modules
import sys
from pathlib import Path

In [ ]:
project_root = str(Path.cwd().resolve().parent)
print(f"Project root: {project_root}")

# Add project root to sys.path for module imports
if project_root not in sys.path:
    sys.path.insert(0, project_root)


# Automatically reload modules before executing code
# This is useful for development to see changes without restarting the kernel.
%load_ext autoreload
# Reload all modules imported with %aimport every time before executing the Python code typed.
%autoreload 1
%aimport src.analysis.vram_usage, src.preprocess.preprocess

In [ ]:
# Import Required Libraries and Modules
from src.analysis.vram_usage import EfficiencyAnalysis
from src.preprocess.preprocess import preprocess_data
from src.database.database_connection import DatabaseConnection

In [ ]:
# Load and Preprocess GPU Job Data
# Connect to the database
db = DatabaseConnection(db_url="../slurm_data.db")

# Query jobs with GPUs
gpu_df = db.connection.query("SELECT * FROM Jobs WHERE GPUs > 0").to_df()

In [ ]:
# Preprocess the data
preprocessed_df = preprocess_data(
    gpu_df, min_elapsed_seconds=0, include_failed_cancelled_jobs=False, include_cpu_only_jobs=True
)

# Initialize the EfficiencyAnalysis class
efficiency_analyzer = EfficiencyAnalysis(df=preprocessed_df, table_name="Jobs")

In [ ]:
import numpy as np

jobs = efficiency_analyzer.filter_jobs_for_analysis(
    gpu_count_filter=1,
    vram_constraint_filter=None,
    allocated_vram_filter={"min": 0, "max": np.inf, "inclusive": False},  # Filter jobs with allocated VRAM > 0
    gpu_mem_usage_filter={"min": 0, "max": np.inf, "inclusive": False},  # Filter jobs with GPU memory usage > 0 GB
)

# Calculate job efficiency metrics
job_metrics = efficiency_analyzer.calculate_job_efficiency_metrics(filtered_jobs=jobs)

# Find inefficient users
users = efficiency_analyzer.calculate_user_efficiency_metrics()

inefficient_users = efficiency_analyzer.find_inefficient_users_by_alloc_vram_efficiency(
    efficiency_threshold=0.3, min_jobs=5
)
display(inefficient_users)

# Filter out users with zero efficiency
inefficient_users = inefficient_users[inefficient_users["expected_value_alloc_vram_efficiency"] > 0]

# Select the top 5 problematic users
top_problematic_users = inefficient_users.head(5)["User"].tolist()

inefficient_users.head(5)

## plot_vram_efficiency

### Annotation Styles Available:
1. **"hover"** - Clean annotations with just GPU hours in small boxes
2. **"combined"** - Both GPU hours and job counts in compact format  
3. **"table"** - Clean plot with detailed metrics displayed in a table below
4. **"none"** - No annotations on the plot
5. **Secondary Y-axis** - Job counts plotted on a secondary y-axis

In [ ]:
users_to_analyze = top_problematic_users

In [ ]:
# Clean plot with detailed metrics table below
print("=== Table Style ===")

fig = efficiency_analyzer.plot_vram_efficiency(
    users=users_to_analyze[:4],
    time_unit="Months",
    days_back=365,
    annotation_style="table",  # This will show a detailed table below the plot
    remove_zero_values=True,
)

In [ ]:
# Secondary Y-axis for job counts (dual-metric visualization)
print("=== Secondary Y-Axis for Job Counts ===")

fig = efficiency_analyzer.plot_vram_efficiency(
    users=users_to_analyze[:4],
    time_unit="Months",
    days_back=180,
    annotation_style="none",  # No text annotations to keep plot clean
    show_secondary_y=True,  # Show job counts on secondary y-axis
    remove_zero_values=True,
)

In [ ]:
# Interactive plot with tooltips (requires plotly)
print("=== Interactive Plot with Tooltips ===")
print("This will fall back to table view if plotly is not installed")

fig = efficiency_analyzer.plot_vram_efficiency_interactive(
    users=users_to_analyze[:4], time_unit="Months", days_back=365, remove_zero_values=True, exclude_fields=["Time"]
)

In [ ]:
# Interactive VRAM Hours plot for the same users
print("=== Interactive VRAM Hours Plot ===")
fig = efficiency_analyzer.plot_vram_hours_interactive(
    users=users_to_analyze[:4], time_unit="Months", days_back=365, max_points=12, exclude_fields=["Time"]
)